In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("TunningTrain").getOrCreate()

23/12/28 15:57:11 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/28 15:57:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 15:57:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/28 15:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/28 15:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
iris = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/iris.csv", header=True, inferSchema=True, sep=",")
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [3]:
from pyspark.ml.feature import VectorAssembler

vector = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")

In [4]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class", outputCol="label")

In [5]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp=MultilayerPerceptronClassifier(maxIter=100, layers=[4,5,4,3], featuresCol="independente", labelCol="label")

In [6]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vector, indexer, mlp])

In [7]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

performance = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000]).addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]]).build()
trainval = TrainValidationSplit(estimator=pipeline, estimatorParamMaps=grid, evaluator=performance, trainRatio=0.8)

In [8]:
model = trainval.fit(irisTreino)

23/12/28 16:00:04 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.75
23/12/28 16:00:04 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed
23/12/28 16:00:18 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.75
23/12/28 16:00:18 ERROR LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search zoom failed


In [9]:
previsao = model.transform(irisTeste)
previsao.select("rawprediction", "probability", "prediction").show()

+--------------------+--------------------+----------+
|       rawprediction|         probability|prediction|
+--------------------+--------------------+----------+
|[-0.6728798167647...|[0.18354576258002...|       2.0|
|[-0.6774951142535...|[0.19478251960914...|       2.0|
|[-0.6761756251858...|[0.18992362234160...|       2.0|
|[-0.6806042946703...|[0.19939188841754...|       2.0|
|[-0.6783243632842...|[0.19316043515955...|       2.0|
|[-0.6820735423309...|[0.20184546196085...|       2.0|
|[-0.6804494926507...|[0.19719939204610...|       2.0|
|[-0.6795403862739...|[0.19420092136346...|       2.0|
|[-0.6840884494266...|[0.20460968126693...|       2.0|
|[-0.6769501048209...|[0.18582104960953...|       2.0|
|[-0.6874543890191...|[0.20993735511957...|       2.0|
|[-0.6763281726564...|[0.18295303007760...|       2.0|
|[-0.6796170535526...|[0.19077307475554...|       2.0|
|[-0.6787938959915...|[0.18914592657537...|       2.0|
|[-0.6781401503057...|[0.18527747519138...|       2.0|
|[-0.63508

In [10]:
performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(f"Acuracia: {acuracia}")

Acuracia: 0.6808510638297872
